In [137]:
#packages used
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, TimeDistributed, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [138]:
#importing data as pandas data frame
DevData = pd.read_json("dev.jsonl", lines = True)
TestData = pd.read_json("test.jsonl", lines = True)
TrainData = pd.read_json("train.jsonl", lines = True)


In [139]:
TrainData

,premise,choice1,choice2,question,label,idx
0,My body cast a shadow over the grass.,The sun was rising.,The grass was cut.,cause,0,0
1,The woman tolerated her friend's difficult beh...,The woman knew her friend was going through a ...,The woman felt that her friend took advantage ...,cause,0,1
2,The women met for coffee.,The cafe reopened in a new location.,They wanted to catch up with each other.,cause,1,2
3,The runner wore shorts.,The forecast predicted high temperatures.,She planned to run along the beach.,cause,0,3
4,The guests of the party hid behind the couch.,It was a surprise party.,It was a birthday party.,cause,0,4
...,...,...,...,...,...,...
345,The photograph faded.,It was fake.,It was old.,cause,1,345
346,The boy got a black eye.,The bully mocked the boy.,The bully punched the boy.,cause,1,346
347,I misplaced my wallet.,I retrieved my phone.,I retraced my steps.,effect,1,347
348,The hunter ran out of ammunition.,He reloaded the gun.,He aimed at the deer.,effect,0,348


In [140]:
#tokenizes test data
TrainTok = Tokenizer(lower=True, split=' ',)

TrainTok.fit_on_texts(TrainData['choice2'])        
TrainTok.fit_on_texts(TrainData['choice1'])
TrainTok.fit_on_texts(TrainData['premise'])


In [141]:
TrainConfig = TrainTok.get_config()

#dictionary of words and indexs
TrainVocab = eval(TrainConfig['index_word'])

#finds length of vocab
TrainVocabSize = len(TrainVocab)

print("there are ", TrainVocabSize, " unique words")

there are  1740  unique words


In [142]:
PremiseTokTrain = TrainTok.texts_to_sequences(TrainData['premise'])
Choice1TokTrain = TrainTok.texts_to_sequences(TrainData['choice1'])
Choice2TokTrain = TrainTok.texts_to_sequences(TrainData['choice2'])

PremisePadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in PremiseTokTrain))
Choice1PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice1TokTrain))
Choice2PadTrain = sequence.pad_sequences(PremiseTokTrain, padding='post', maxlen=max(len(i) for i in Choice2TokTrain))

In [143]:
TrainCat = tf.concat([PremisePadTrain, Choice1PadTrain,Choice2PadTrain], 1)
 

In [146]:
additional_metrics = ['accuracy']
loss_function = BinaryCrossentropy()
number_of_epochs = 5
optimizer = Adam()
validation_split = 0.20
verbosity_mode = 1

# Define the Keras model
model = Sequential()
model.add(Embedding(TrainVocabSize, 64, input_length=35))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=optimizer, loss=loss_function, metrics=additional_metrics)

# Give a summary
model.summary()

# Train the model
history = model.fit(TrainCat, TrainData['label'], epochs=number_of_epochs, verbose=verbosity_mode, validation_split=validation_split)


Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 35, 64)            111360    
_________________________________________________________________
lstm_16 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 65        
Total params: 144,449
Trainable params: 144,449
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5


2021-11-03 11:54:50.227799: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 11:54:50.308442: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 11:54:50.387962: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


9/9 [==============================] - 1s 71ms/step - loss: 0.6938 - accuracy: 0.5036 - val_loss: 0.6928 - val_accuracy: 0.5143


2021-11-03 11:54:50.900751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2021-11-03 11:54:50.933520: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/5
9/9 [==============================] - 0s 32ms/step - loss: 0.6935 - accuracy: 0.5036 - val_loss: 0.6928 - val_accuracy: 0.5143
Epoch 3/5
9/9 [==============================] - 0s 23ms/step - loss: 0.6931 - accuracy: 0.5071 - val_loss: 0.6928 - val_accuracy: 0.5143
Epoch 4/5
9/9 [==============================] - 0s 23ms/step - loss: 0.6930 - accuracy: 0.5071 - val_loss: 0.6928 - val_accuracy: 0.5143
Epoch 5/5
9/9 [==============================] - 0s 29ms/step - loss: 0.6931 - accuracy: 0.5107 - val_loss: 0.6928 - val_accuracy: 0.5143
